<b>
<div style="text-align: center;">
<span style="font-size: 200%;">行動価値関数 (Action Value Function)</span>
</div>
</b>

<br>

<b>
<div style="text-align: right;">
    <span style="font-size: 150%;">
        2019/12/25
    </span><br>
    <span style="font-size: 150%;">
        Masaya Mori
    </span>
</div>
</b>

# 問題設定

![問題設定](figure/ProblemSetting.jpg)

- <span style="font-size: 120%;"> 今回は状態価値関数の時と同様な，5×5の格子世界におけるAgentの行動価値を導出する</span>  
- <span style="font-size: 120%;"> ルールは前回と同様なので，[状態価値関数](https://github.com/rrrrind/reinforcement-learning/blob/master/DP/src/State%20Value%20Function/State_Value_Function.ipynb)の問題設定を参照してください</span>  
- <span style="font-size: 120%;"> 行動価値は，
    $$
    {\rm Q^{\pi}}(s,a) = \Sigma_{s'}{\rm P}^{a}_{s,s'}\bigr[{\rm R}^{a}_{s,s'} + \
    \gamma  \Sigma_{a}{\rm \pi} (s',a') {\rm Q^{\pi}}(s',a') \bigl], \nonumber
    $$
によって算出する</span>  

# マルコフ決定過程

<span style="font-size: 120%;"> こちらも前回と同じ．[状態価値関数](https://github.com/rrrrind/reinforcement-learning/blob/master/DP/src/State%20Value%20Function/State_Value_Function.ipynb)のマルコフ決定過程を参照．
</span>  

# Pythonでの実装

<span style="font-size: 120%;">　エージェントの動作も前回と同じなので，
    <span style="color: red; ">状態価値関数の部分だけを行動価値関数に書き換える</span>．
</span>

In [4]:
import numpy as np
from tqdm.notebook import tqdm_notebook as tqdm
import time

In [5]:
stage = [[0,4],[1,4],[2,4],[3,4],[4,4],
         [0,3],[1,3],[2,3],[3,3],[4,3],
         [0,2],[1,2],[2,2],[3,2],[4,2],
         [0,1],[1,1],[2,1],[3,1],[4,1],
         [0,0],[1,0],[2,0],[3,0],[4,0]]

In [6]:
class Agent(): # stage_map, the array number of the stage    
    
    def __init__(self):
        # 行動a,方策πの宣言。引数で使うので、初めに宣言する。
        self.actions = [[0,1],[0,-1],[-1,0],[1,0]] # up,down,left,right
        self.each_pi = [0.25,0.25,0.25,0.25]
        self.position = []
    
    def get_actions(self):
        # 全行動の取得
        return self.actions
        
    def set_pos(self,now_pos):
        # 現在地の更新
        self.position = now_pos
        
    def get_pos(self):
        # 現在地の取得
        return self.position
    
    def move(self,action):
        # 行動後の位置の取得
        if self.get_pos() == [1,4]: # 10ptゾーン
            next_pos = [1,0]
        elif self.get_pos() == [3,4]: # 5ptゾーン
            next_pos = [3,2]
        else :
            next_pos = [self.get_pos()[0] + action[0], self.get_pos()[1] + action[1]]
        
        # 境界線外に出ている時の処理
        if 0 > next_pos[0] or next_pos[0] > 4 or 0 > next_pos[1] or next_pos[1] > 4:
            next_pos = self.get_pos()
            
        self.set_pos(next_pos)
    
    def pi(self,state,num):
        return self.each_pi[num]
        
    def reward(self,state,action):
        if state == [1,4]:
            return 10
        
        if state == [3,4]:
            return 5
        
        if state[1] == 4 and action == [0,1]:
            return -1
        elif state[1] == 0 and action == [0,-1]:
            return -1
        elif state[0] == 0 and action == [-1,0]:
            return -1
        elif state[0] == 4 and action == [1,0]:
            return -1
        else :
            return 0
    
    
#------------------------------------  以下を行動価値関数に書き換える  ------------------------------------

    def q_pi(self,state,act,n,out,iter_num): # この"state"は，潜った時の位置を記憶しておくため
        if n == iter_num :
            for i, action in enumerate(self.actions):
                # 最下層での処理．最下層では前後左右に動いた際の報酬を一気に計算する．
                out += self.pi(state,i) * self.reward(state,action)
            return out
        else :
            # 行動を1つ選択した後の報酬なのでfor文の外に記述する
            # (以下のfor文は4回，つまり上下左右の報酬を求めるためのループ文)
            out += self.reward(self.get_pos(),act)
            self.move(act)
            position = self.get_pos()
            for i, action in enumerate(self.actions):
            # 行動した後は方策に従う，つまり上下左右の報酬を求める．
                out += 0.9 * self.pi(self.get_pos(),i) * self.q_pi(self.get_pos(), action, n+1, 0, iter_num)
                self.set_pos(position) # "self.get_pos()"から1つ前の状態"state"に戻るため
            return out

<span style="font-size: 120%;"> 60行目の${\rm \pi}(s,a)$に$\gamma$を掛けていないが，これは${\rm  Q^{\pi}}(s,a)$によって得られる報酬に対して時間割引$\gamma$が掛けられていたので，再帰をしない，つまり${\rm  Q^{\pi}}(s,a)$がなくなると$\gamma$も同じくなくなると私は考えている(合ってるかどうかは分からない)．
<span>

In [7]:
agent = Agent()
q_pi_value = np.zeros([25,4])

start = time.time()
for i,locate in tqdm(enumerate(stage)):
    for j, action in enumerate(agent.get_actions()):
        agent.set_pos(locate)
        q_pi_value[i][j] = agent.q_pi(agent.get_pos(),action,0,0,10)
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")


elapsed_time:551.5425078868866[sec]


In [8]:
outcome = np.array(q_pi_value)
#outcome = outcome.reshape(5,5)
outcome

array([[ 2.00449195,  1.34879984,  2.00449195,  8.01656136],
       [ 8.87490074,  8.87490074,  8.87490074,  8.87490074],
       [ 2.99431619,  1.99623722,  8.01656136,  4.770906  ],
       [ 5.29951596,  5.29951596,  5.29951596,  5.29951596],
       [ 0.24642576,  0.41080755,  4.770906  ,  0.24642576],
       [ 3.00449195,  0.03079759,  0.34879984,  2.70058495],
       [ 8.01656136,  0.64937268,  1.34879984,  1.99623722],
       [ 3.99431619,  0.60480828,  2.70058495,  1.66493039],
       [ 4.770906  ,  0.29951596,  1.99623722,  0.41080755],
       [ 1.24642576, -0.38440394,  1.66493039, -0.58919245],
       [ 1.34879984, -0.84691265, -0.96920241,  0.64937268],
       [ 2.70058495, -0.34432625,  0.03079759,  0.60480828],
       [ 1.99623722, -0.27917008,  0.64937268,  0.29951596],
       [ 1.66493039, -0.46541157,  0.60480828, -0.38440394],
       [ 0.41080755, -1.00787557,  0.29951596, -1.38440394],
       [ 0.03079759, -1.59016659, -1.84691265, -0.34432625],
       [ 0.64937268, -1.

# 結果・考察

![結果](figure/outcome.jpg)

- <span style="font-size: 120%;"> 4方向への行動確率が同じ( つまり$\pi(s,a)=0.25$ の)場合における行動価値を算出した </span>
- <span style="font-size: 120%;"> 上の図は，再帰を10回行ったときの結果である</span>
- <span style="font-size: 120%;"> 結果として，基本『up』の確率を高くすれば，方策がより良くなるということが分かった</span>
- <span style="font-size: 120%;"> また再帰を行うと，行動価値を求めるのに莫大な時間が掛かるということも分かった(10分弱)</span>
- <span style="font-size: 120%;"> したがって，再帰の代わりに行動価値関数を近似する等して，時間短縮を図る必要があると考えた</span>

# 参考資料

- [今さら聞けない強化学習（3）：行動価値関数とBellman方程式](https://qiita.com/triwave33/items/8966890701169f8cad47)
- [今さら聞けない強化学習（4）：行動価値関数の実装](https://qiita.com/triwave33/items/669a975b74461559addc)